<a href="https://colab.research.google.com/github/kmk4444/Retrieval-augmented-generation/blob/main/Part8_hybrid_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Requirements.txt**

In [5]:
!touch requirements.txt
!echo langchain >> requirements.txt
!echo langchain-openai >> requirements.txt
!echo openai >> requirements.txt
!echo langchain-google-genai >> requirements.txt
!echo cohere >> requirements.txt
!echo faiss-cpu >> requirements.txt
!echo streamlit >> requirements.txt
!echo python-dotenv >> requirements.txt
!echo llama-index >> requirements.txt
!echo pypdf >> requirements.txt
!echo chromadb >> requirements.tx
!echo beautifulsoup4 >> requirements.tx
!echo matplotlib >> requirements.tx
!echo rank_bm25 >> requirements.tx
!echo replicate >> requirements.txt

In [13]:
pip install rank_bm25

**Bash/command**

In [6]:
pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.3/320.3 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 87.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 53.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.0/121.0 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 39.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━

In [14]:
%%writefile hybridhelper.py
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_community.retrievers import BM25Retriever
from langchain.retrievers import EnsembleRetriever
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain_community.document_loaders import WebBaseLoader
import os
from dotenv import load_dotenv

#load_dotenv()

#my_key_openai = os.getenv("openai_apikey")
my_key_openai="----"
embeddings = OpenAIEmbeddings(api_key=my_key_openai)

doc_list = [
    "I like apples",
    "I like oranges",
    "Apples and oranges are fruits",
    "I like computers by Apple",
    "I love fruit juice but particularly apples as apples are the best",
    "Air Cana serves apple juice",
    "Beetlejuice is a terrible movie",
    "That country literally is a banana republic",
    "The iPhone made its manufacturer rich",
    "I dislike apples",
]

def load_and_split_documents(target_url):
  loader = WebBaseLoader(target_url)

  raw_documents = loader.load()

  text_splitter = RecursiveCharacterTextSplitter(
      chunk_size=1000,
      chunk_overlap=0,
      length_function=len
  )

  splitted_documents = text_splitter.split_documents(raw_documents)

  custom_documents = []

  for i, raw_doc in enumerate(splitted_documents):

    new_doc = Document(
        page_content = raw_doc.page_content,
        metadata = {
            "source": raw_doc.metadata["source"],
            "title" : raw_doc.metadata["title"],
            "description": raw_doc.metadata["description"],
            "language" : raw_doc.metadata["language"],
            "doc_id" : i
        }
    )

    custom_documents.append(new_doc)

  return custom_documents

def get_relevant_documents_with_bm25(documents, query):
  bm25_retriever  = BM25Retriever.from_documents(documents=documents)
  bm25_retriever.k=4

  bm25_relevant_documents = bm25_retriever.get_relevant_documents(query=query)

  return bm25_relevant_documents,bm25_retriever

def get_relevant_documents_with_FAISS(documents, query):

  vectorstore = FAISS.from_documents(documents,embeddings)
  faiss_retriever = vectorstore.as_retriever(search_kwargs = {"k":4})

  FAISS_relevant_documents = faiss_retriever.get_relevant_documents(query)

  return FAISS_relevant_documents, faiss_retriever

def get_relevant_documents_for_hybrid_search(query, retriever1, retriever2, weight1=0.5, weight2=0.5):

  ensemble_retriever = EnsembleRetriever(
      retrievers=[retriever1, retriever2],
      weights=[weight1,weight2]
  )

  hybrid_relevant_documents = ensemble_retriever.get_relevant_documents(query)

  return hybrid_relevant_documents


Overwriting hybridhelper.py


In [15]:
%%writefile hybrid_search.py
import streamlit as st
import hybridhelper

st.set_page_config(layout="wide")
st.title("Advanced RAG: Hybrid Search | Hibrit Arama")
st.divider()

col_left, col_input, col_right = st.columns(3)

with col_left:
  st.empty()

with col_input:
  target_url = st.text_input(label="Hedef Web Adresini Giriniz", value="https://cbarkinozer.medium.com/reg%C3%BCle-edilmemi%C5%9F-yapay-zeka-teknolojileri-kullanman%C4%B1n-tehlikeleri-nelerdir-fa465da15491")
  original_prompt = st.text_input(label="Sorunuzu Giriniz:", value="Yapay zekayla ilgili muhtemel sorunları çözmek için yapılabilecek aksiyonlar nelerdir?")
  weight_options = ["%90 Karakter Bazlı", "%75 Karakter Bazlı", "%50 - %50", "%75 Semantik Bazlı", "%90 Semantik Bazlı"]
  retriever_weight = st.select_slider(label="Arama Yöntemlerinin Ağırlıklarını Belirleyin:",options=weight_options, value="%50 - %50")

  submit_btn = st.button(label="Gönder")
  st.divider()

with col_right:
  st.empty()

col_keyword, col_hybrid, col_semantic = st.columns(3)

with col_keyword:
    st.subheader("Karakter Bazlı Arama | BM25")
    st.divider()

with col_hybrid:
    st.subheader("Hibrit Arama")
    st.divider()

with col_semantic:
    st.subheader("Semantik Arama")
    st.divider()

if submit_btn:

  initial_documents = hybridhelper.load_and_split_documents(target_url=target_url)

  bm25_documents, bm25retriever = hybridhelper.get_relevant_documents_with_bm25(documents=initial_documents, query=original_prompt)

  faiss_documents, faissretriever = hybridhelper.get_relevant_documents_with_FAISS(documents=initial_documents, query=original_prompt)

  weight1 = 0.5
  if retriever_weight == "%90 Karakter Bazlı":
      weight1 = 0.9
  elif retriever_weight == "%75 Karakter Bazlı":
      weight1 = 0.75
  elif retriever_weight == "%50 - %50 Dengeli":
      weight1 = 0.5
  elif retriever_weight == "%75 Semantik Bazlı":
      weight1 = 0.25
  elif retriever_weight == "%90 Semantik Bazlı":
      weight1 = 0.1

  hybrid_search_documents = hybridhelper.get_relevant_documents_for_hybrid_search(
                                                        query=original_prompt,
                                                        retriever1=bm25retriever,
                                                        retriever2=faissretriever,
                                                        weight1=weight1,
                                                        weight2=1-weight1
                                                        )

  for keyword_doc in bm25_documents:
      col_keyword.warning(f"ID: {keyword_doc.metadata['doc_id']} || {keyword_doc.page_content}")

  for faiss_doc in faiss_documents:
      col_semantic.info(f"ID: {faiss_doc.metadata['doc_id']} || {faiss_doc.page_content}")

  for hybrid_doc in hybrid_search_documents:
      col_hybrid.success(f"ID: {hybrid_doc.metadata['doc_id']} || {hybrid_doc.page_content}")


Overwriting hybrid_search.py


In [17]:
!npm install localtunnel
!streamlit run /content/hybrid_search.py &>/content/logs.txt &
!npx localtunnel --port 8501

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
updated 1 package and audited 36 packages in 0.492s

3 packages are looking for funding
  run `npm fund` for details

found 2 moderate severity vulnerabilities
  run `npm audit fix` to fix them, or `npm audit` for details
npx: installed 22 in 1.937s
your url is: https://eager-dryers-thank.loca.lt
/root/.npm/_npx/24620/lib/node_modules/localtunnel/bin/lt.js:81
    throw err;
    ^

Error: connection refused: localtunnel.me:38485 (check your firewall settings)
    at Socket.<anonymous> (/root/.npm/_npx/24620/lib/node_modules/localtunnel/lib/TunnelCluster.js:52:11)
    at Socket.emit (events.js:315:20)
    at emitErrorNT (internal/streams/destroy.js:106:8)
    at emitErrorC